# Scope

In this post, we demonstrate how to optimize the hyperparameters of a support vector machine (SVM).
We are using the `r mlr_pkg("mlr3")` machine learning framework with the `r mlr_pkg("mlr3tuning")` extension package.

First, we start by showing the basic building blocks of `r mlr_pkg("mlr3tuning")`  and tune the `cost` and `gamma` hyperparameters of an SVM with a radial basis function on the [Iris data set](https://mlr3.mlr-org.com/reference/mlr_tasks_iris.html).
After that, we use transformations to tune the `cost` hyperparameter on the logarithmic scale.
Next, we explain the importance of dependencies to tune hyperparameters like `degree` which are dependent on the choice of kernel.
After that, we fit an SVM with optimized hyperparameters on the full dataset.
Finally, nested resampling is used to compute an unbiased performance estimate of our tuned SVM.

# Prerequisites

We load the `r mlr_pkg("mlr3verse")`  package which pulls in the most important packages for this example.

In [ ]:
library(mlr3verse)

We initialize the random number generator with a fixed seed for reproducibility, and decrease the verbosity of the logger to keep the output clearly represented.
The [`lgr`](https://mlr3book.mlr-org.com/logging.html) package is used for logging in all `r mlr_pkg("mlr3")` packages.
The `r mlr_pkg("mlr3")` logger prints the logging messages from the base package, whereas the `r mlr_pkg("bbotk")`  logger is responsible for logging messages from the optimization packages (e.g. `r mlr_pkg("mlr3tuning")` ).

In [ ]:
set.seed(7832)
lgr::get_logger("mlr3")$set_threshold("warn")
lgr::get_logger("bbotk")$set_threshold("warn")

In the example, we use the [Iris data set](https://mlr3.mlr-org.com/reference/mlr_tasks_iris.html) which classifies 150 flowers in three species of Iris.
The flowers are characterized by sepal length and width and petal length and width.
The Iris data set allows us to quickly fit models to it.
However, the influence of hyperparameter tuning on the predictive performance might be minor.
Other data sets might give more meaningful tuning results.

In [ ]:
# retrieve the task from mlr3
task = tsk("iris")

# generate a quick textual overview using the skimr package
skimr::skim(task$data())

We choose the support vector machine implementation from the `r cran_pkg("e1071")` package (which is based on [LIBSVM](https://www.csie.ntu.edu.tw/~cjlin/libsvm/)) and use it as a classification machine by setting `type` to `"C-classification"`.

In [ ]:
learner = lrn("classif.svm", type = "C-classification", kernel = "radial")

# Tuning Search Space

For tuning, it is important to create a search space that defines the type and range of the hyperparameters.
A learner stores all information about its hyperparameters in the slot `$param_set`.
Not all parameters are tunable. 
We have to choose a subset of the hyperparameters we want to tune.

In [ ]:
print(learner$param_set)

We use the `r ref("to_tune()")` function to define the range over which the hyperparameter should be tuned.
We opt for the `cost` and `gamma` hyperparameters of the `radial` kernel and set the tuning ranges with lower and upper bounds.

In [ ]:
learner$param_set$values$cost = to_tune(0.1, 10)
learner$param_set$values$gamma = to_tune(0, 5)

# Tuning

We specify how to evaluate the performance of the different hyperparameter configurations.
For this, we choose 3-fold cross validation as the resampling strategy and the classification error as the performance measure.

In [ ]:
resampling = rsmp("cv", folds = 3)
measure = msr("classif.ce")

Usually, we have to select a budget for the tuning.
This is done by choosing a `r ref("Terminator")`, which stops the tuning e.g. after a performance level is reached or after a given time.
However, some tuners like grid search terminate themselves.
In this case, we  choose a terminator that never stops and the tuning is not stopped before all grid points are evaluated.

In [ ]:
terminator = trm("none")

At this point, we can construct a `r ref("TuningInstanceSingleCrit")` that describes the tuning problem.

In [ ]:
instance = TuningInstanceSingleCrit$new(
  task = task,
  learner = learner,
  resampling = resampling,
  measure = measure,
  terminator = terminator
)

print(instance)

Finally, we have to choose a `r ref("Tuner")`.
[Grid Search](https://mlr3tuning.mlr-org.com/reference/mlr_tuners_grid_search.html) discretizes numeric parameters into a given resolution and constructs a grid from the Cartesian product of these sets. 
Categorical parameters produce a grid over all levels specified in the search space.
In this example, we only use a resolution of 5 to keep the runtime low.
Usually, a higher resolution is used to create a denser grid.

In [ ]:
tuner = tnr("grid_search", resolution = 5)

print(tuner)

We can preview the proposed configurations by using `r ref("generate_design_grid()")`. This function is internally executed by `r ref("TunerGridSearch")`.

In [ ]:
generate_design_grid(learner$param_set$search_space(), resolution = 5)

We trigger the tuning by passing the `r ref("TuningInstanceSingleCrit")` to the `$optimize()` method of the `r ref("Tuner")`. The instance is modified in-place.

In [ ]:
instance = readRDS("data/instance_1.rda")

In [ ]:
tuner$optimize(instance)

We plot the performances depending on the evaluated `cost` and `gamma` values.

In [ ]:
autoplot(instance, type = "surface", cols_x = c("cost", "gamma"), learner = lrn("regr.km"))

In [ ]:
# regr.km prints a log
log = capture.output(autoplot(instance, type = "surface", cols_x = c("cost", "gamma"), learner = lrn("regr.km")))

The points mark the evaluated `cost` and `gamma` values.
We should not infer the performance of new values from the heatmap since it is only an interpolation.
However, we can see the general interaction between the hyperparameters.

Tuning a learner can be shortened by using the `r ref("tune()")`-shortcut.

In [ ]:
learner = lrn("classif.svm", type = "C-classification", kernel = "radial")
learner$param_set$values$cost = to_tune(0.1, 10)
learner$param_set$values$gamma = to_tune(0, 5)

instance = tune(
  method = "grid_search", 
  task = tsk("iris"), 
  learner = learner, 
  resampling = rsmp ("holdout"), 
  measure = msr("classif.ce"),
  resolution = 5
)

# Transformation

Next, we want to tune the `cost` and `gamma` hyperparameter more efficiently.
It is recommended to tune `cost` and `gamma` on the logarithmic scale [@hsuPracticalGuideSupport2003].
The log transformation emphasizes smaller `cost` and `gamma` values but also creates large values.
Therefore, we use a log transformation to emphasize this region of the search space with a denser grid.

Generally speaking, transformations can be used to convert hyperparameters to a new scale.
These transformations are applied before the proposed configuration is passed to the `r ref("Learner")`.
We can directly define the transformation in the `r ref("to_tune()")` function.
The lower and upper bound is set on the original scale.

In [ ]:
learner = lrn("classif.svm", type = "C-classification", kernel = "radial")

# tune from 2^-15 to 2^15 on a log scale
learner$param_set$values$cost = to_tune(p_dbl(-15, 15, trafo = function(x) 2^x))

# tune from 2^-15 to 2^5 on a log scale
learner$param_set$values$gamma = to_tune(p_dbl(-15, 5, trafo = function(x) 2^x))

Transformations to the log scale are the ones most commonly used. 
We can use a shortcut for this transformation.
The lower and upper bound is set on the transformed scale.

In [ ]:
learner$param_set$values$cost = to_tune(p_dbl(1e-5, 1e5, logscale = TRUE))
learner$param_set$values$gamma = to_tune(p_dbl(1e-5, 1e5, logscale = TRUE))

We create a new `r ref("TuningInstanceSingleCrit")` and trigger the tuning.

In [ ]:
instance = readRDS("data/instance_2.rda")

In [ ]:
instance = tune(
  method = "grid_search", 
  task = task, 
  learner = learner, 
  resampling = resampling, 
  measure = measure,
  resolution = 5
)

The hyperparameter values after the transformation are stored in the `x_domain` column as lists.
We can expand these lists into multiple columns by using `as.data.table()`.
The hyperparameter names are prefixed by `x_domain`.

In [ ]:
data = as.data.table(instance$archive)
data[, .(cost, gamma, x_domain_cost, x_domain_gamma)]

We plot the performances depending on the evaluated `cost` and `gamma` values.

In [ ]:
library(ggplot2)
library(scales)
autoplot(instance, type = "points", cols_x = c("x_domain_cost", "x_domain_gamma")) +
  scale_x_continuous(
    trans = log2_trans(),
    breaks = trans_breaks("log10", function(x) 10^x),
    labels = trans_format("log10", math_format(10^.x))) +
  scale_y_continuous(
    trans = log2_trans(),
    breaks = trans_breaks("log10", function(x) 10^x),
    labels = trans_format("log10", math_format(10^.x)))

# Dependencies

Dependencies ensure that certain parameters are only proposed depending on values of other hyperparameters.
We want to tune the `degree` hyperparameter that is only needed for the `polynomial` kernel.

In [ ]:
learner = lrn("classif.svm", type = "C-classification")

learner$param_set$values$cost = to_tune(p_dbl(1e-5, 1e5, logscale = TRUE))
learner$param_set$values$gamma = to_tune(p_dbl(1e-5, 1e5, logscale = TRUE))

learner$param_set$values$kernel = to_tune(c("polynomial", "radial"))
learner$param_set$values$degree = to_tune(1, 4)

The dependencies are already stored in the learner parameter set.

In [ ]:
learner$param_set$deps

The `gamma` hyperparameter depends on the kernel being `polynomial`, `radial` or `sigmoid`

In [ ]:
learner$param_set$deps$cond[[5]]

whereas the `degree` hyperparameter is solely used by the `polynomial` kernel.

In [ ]:
learner$param_set$deps$cond[[3]]

We preview the grid to show the effect of the dependencies.

In [ ]:
generate_design_grid(learner$param_set$search_space(), resolution = 2)

The value for `degree` is `NA` if the dependency on the `kernel` is not satisfied.

We create a new `r ref("TuningInstanceSingleCrit")` and trigger the tuning.

In [ ]:
instance = readRDS("data/instance_3.rda")

In [ ]:
instance = tune(
  method = "grid_search", 
  task = task, 
  learner = learner, 
  resampling = resampling, 
  measure = measure,
  resolution = 3
)

In [ ]:
instance$result

# Final Model

We add the optimized hyperparameters to the learner and train the learner on the full dataset.

In [ ]:
learner = lrn("classif.svm")
learner$param_set$values = instance$result_learner_param_vals
learner$train(task)

The trained model can now be used to make predictions on new data.
A common mistake is to report the performance estimated on the resampling sets on which the tuning was performed (`instance$result_y`) as the model's performance.
These scores might be biased and overestimate the ability of the fitted model to predict with new data.
Instead, we have to use nested resampling to get an unbiased performance estimate.

# Nested Resampling

Tuning should not be performed on the same resampling sets which are used for evaluating the model itself, since this would result in a biased performance estimate.
[Nested resampling](https://mlr3book.mlr-org.com/nested-resampling.html) uses an outer and inner resampling to separate the tuning from the performance estimation of the model.
We can use the `r ref("AutoTuner")` class for running nested resampling.
The `r ref("AutoTuner")` wraps a `r ref("Learner")` and tunes the hyperparameter of the learner during `$train()`.
This is our inner resampling loop.

In [ ]:
learner = lrn("classif.svm", type = "C-classification")
learner$param_set$values$cost = to_tune(p_dbl(1e-5, 1e5, logscale = TRUE))
learner$param_set$values$gamma = to_tune(p_dbl(1e-5, 1e5, logscale = TRUE))
learner$param_set$values$kernel = to_tune(c("polynomial", "radial"))
learner$param_set$values$degree = to_tune(1, 4)

resampling_inner = rsmp("cv", folds = 3)
terminator = trm("none")
tuner = tnr("grid_search", resolution = 3)

at = AutoTuner$new(
  learner = learner,
  resampling = resampling_inner,
  measure = measure,
  terminator = terminator,
  tuner = tuner,
  store_models = TRUE)

We put the `r ref("AutoTuner")` into a `r ref("resample()")` call to get the outer resampling loop.

In [ ]:
rr = readRDS("data/rr_1.rda")

In [ ]:
resampling_outer = rsmp("cv", folds = 3)
rr = resample(task = task, learner = at, resampling = resampling_outer, store_models = TRUE)

We check the inner tuning results for stable hyperparameters.
This means that the selected hyperparameters should not vary too much.
We might observe unstable models in this example because the small data set and the low number of resampling iterations might introduce too much randomness.
Usually, we aim for the selection of stable hyperparameters for all outer training sets.

In [ ]:
extract_inner_tuning_results(rr)

Next, we want to compare the predictive performances estimated on the outer resampling to the inner resampling (`extract_inner_tuning_results(rr)`).
Significantly lower predictive performances on the outer resampling indicate that the models with the optimized hyperparameters overfit the data.

In [ ]:
rr$score()

The archive of the `r ref("AutoTuner")`s allows us to inspect all evaluated hyperparameters configurations with the associated predictive performances.

In [ ]:
rr$learners[[1]]$archive

The aggregated performance of all outer resampling iterations is essentially the unbiased performance of an SVM with optimal hyperparameter found by grid search.

In [ ]:
rr$aggregate()

Applying nested resampling can be shortened by using the `r ref("tune_nested()")`-shortcut.

In [ ]:
learner = lrn("classif.svm", type = "C-classification")
learner$param_set$values$cost = to_tune(p_dbl(1e-5, 1e5, logscale = TRUE))
learner$param_set$values$gamma = to_tune(p_dbl(1e-5, 1e5, logscale = TRUE))
learner$param_set$values$kernel = to_tune(c("polynomial", "radial"))
learner$param_set$values$degree = to_tune(1, 4)

rr = tune_nested(
  method = "grid_search",
  task = tsk("iris"),
  learner = learner, 
  inner_resampling = rsmp ("cv", folds = 3),
  outer_resampling = rsmp("cv", folds = 3), 
  measure = msr("classif.ce"),
  resolution = 3
)

# Resources

The [mlr3book](https://mlr3book.mlr-org.com/) includes chapters on [tuning spaces](https://mlr3book.mlr-org.com/searchspace.html) and [hyperparameter tuning](https://mlr3book.mlr-org.com/tuning.html). 
The [mlr3cheatsheets](https://cheatsheets.mlr-org.com/) contain frequently used commands and workflows of mlr3.

